# Валидация данных с помощью Pydantic

Pydantic - стороняя библиотека языка python для проверки данных и их приведения к нужному типу. Используя валидацию данных можно сделать работу приложения более стабильной.

## Пример

Есть Cool Service с интересной фичей. Остальные сервисы не хотят ее реализовывать, но хотят интегрироваться. 

У User должен быть uuid и прочие данные. Первые два сервиса передают правильные данные, но третий сервис вместо uuid передает строку, которая не  является uuid. Из-за этого наш сервис падает. Если бы мы использовали валидацию входных данных, то вывели бы ошибку и наш сервис не упал. 
<img src="./images/cool-service.png" width=400>

***

## Почему лучше использовать pydantic для валидации данных?

Для того, чтобы понять почему pydantic удобен для валидации данных рассмотрим другие инструменты для валидации данных в python.


### 1. dataclass
В Python есть dataclass, но он не делает валидирование типов. Т.е. в dataclass можно указать типы каждой переменной, но при создании объекта не будет никакой проверки для значений переменных. 

In [1]:
from dataclasses import dataclass


@dataclass
class User:
    uuid: str
    name: str

In [2]:
user = User(uuid=10, name=10)
type(user.uuid), type(user.name)

(int, int)

__________________________________

### 2. Существующие библиотеки для валидации данных

Самые популярные библиотеки валидирующие данные для языка Python:
* Cerberus 2.6k ⭐, Voluptuous 1.7k ⭐, Schema 2.5k ⭐ (работа с этими библиотеками похожа)
* Marshmallow 5.9k ⭐
* Pydantic 8.7k ⭐

***

## Сerberus

Предоставляет функционал для валидализации данных

In [1]:
!pip install Cerberus

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


Для валидации необходимо отпределить схему валидализации и передать ее в класс Validator. Схема валидатора является mapping, чаще всего задается в виде стандартного dict

In [32]:
from cerberus import Validator

In [33]:
user_validator = Validator(
    {
        'name': {'type': 'string'},
        'email': {'type': 'string', 'required': True},
        'age': {'type': 'integer', 'min': 0, 'max': 150}
    }
)

In [34]:
user = {
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'age': 30,
}
current_user = user_validator.validate(user)
current_user

True

### Плюсы и минусы
    ✅ Выполняется валидация данных
    ❌ Свой собственный язык данных (str -> string, int -> integer)
    ❌ Тяжело описать сложную схему с помощью dict
    ❌ Сложные схемы проще описать в YAML файле
    ❌ Выполняет только валидацию данных, не возвращает объект экземпляр класса

***

## marshmallow

In [4]:
!pip install marshmallow

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


Схема описывается в виде класса с помощью типов из собственного модуля fields.

In [6]:
from marshmallow import Schema, fields, validate


class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email(required=True)
    age = fields.Int()

In [7]:
user = {
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'age': 30,
}
schema = UserSchema()
schema.dump(user)

{'age': 30, 'name': 'John Doe', 'email': 'john.doe@example.com'}

Необходимо реализовать еще один класс, чтобы получить экземпляр класса

In [11]:
from marshmallow import Schema, fields, validate, post_load, post_dump


class User:
    def __init__(self, name, email, age):
        self.name = name
        self.email = email
        self.age = age
        
    def __repr__(self):
        return f'name={self.name}, email={self.email}, age={self.age}'
        

class UserSchema(Schema):
    name = fields.Str()
    email = fields.Email(required=True)
    age = fields.Int()
    
    @post_dump
    def make_user(self, data, **kwargs):
        return User(**data)

In [12]:
user = {
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'age': 30,
}
schema = UserSchema()
schema.dump(user)

name=John Doe, email=john.doe@example.com, age=30

Можно использовать вложенные схемы и структуры данных

In [53]:
from marshmallow import Schema, fields, validate


class BookSchema(Schema):
    title = fields.String(required=True, validate=validate.Length(max=120))
    isbn = fields.String(required=True)

class UserSchema(Schema):
    email = fields.Email(required=True)
    books = fields.Nested(BookSchema, many=True, required=False)

### Плюсы и минусы

    ✅ Схема задается в виде класса
    ✅ Нет необходимости изучать дополнительный язык типов данных. Все типы данных содержатся в fields и любая ide может их подсказать
    ✅ Можно возращать валидированный экземпляр класса
    ❌ Если нужно вернуть экземпляр класса, то необходимо реализовать два класса

***

# pydantic

In [2]:
!pip install pydantic

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


Объекты для валидации описываются с помощью классов, унаследованные от  класса BaseModel из модуля pydantic, который возвращает экземпляр класса. Для описание типов используются стандартные типы языка Python.

In [42]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    email: str
    age: int

In [43]:
user = {
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'age': 30,
}
User(**user)

User(name='John Doe', email='john.doe@example.com', age=30)

При использовании pydantic:

    ✅ Схема задается в виде класса
    ✅ Нет необходимости изучать дополнительный язык типов данных. Используется стандартные типы языка Python
    ✅ Можно возращать валидированный экземпляр класса
    ✅ Если нужно вернуть экземпляр класса, то нет необходимости реализовать дополнительный класс

***

### Легкий способ перейти от python dataclass к валидации с помощью pydantic

In [13]:
from dataclasses import dataclass

@dataclass
class User:
    name: str
    email: str
    age: int

        
User(email='john.doe@example.com', age='wrong data', name='John Doe')

User(name='John Doe', email='john.doe@example.com', age='wrong data')

Переменная **age** дожна быть типа **int**, мы передали в эту переменную **str** и не было выведено ошибки

***

Для того чтобы сделать валидацию необходимо использовать pydantic dataclass, для этого импортируем dataclass из pydantic.dataclasses.

Укажем значение переменной **age='wrong data'** и pydantic выведет ошибку:

In [15]:
from pydantic.dataclasses import dataclass

@dataclass
class User:
    name: str
    email: str
    age: int

User(email='john.doe@example.com', age='wrong data', name='John Doe')

ValidationError: 1 validation error for User
age
  value is not a valid integer (type=type_error.integer)

Укажем корректные данные:

In [16]:
User(email='john.doe@example.com', age=30, name='John Doe')

User(name='John Doe', email='john.doe@example.com', age=30)

Таким образом заменив одну строку импорта в программе, можно легко добавить валидацию данных в уже существующих проектах, где используются python dataclass

***

### Описание вложенных объектов данных

Для описания вложенного объекта данных необходимо указать, что типом данных этой переменной является другая модель.

In [50]:
from pydantic import BaseModel
from typing import List

class Book(BaseModel):
    name: str
    isbn: int
        

class User(BaseModel):
    name: Optional[str]
    email: str
    books: List[Book]

        
User(**{
    'name': 'Isaac Asimov',
    'email': 'isaac.asimov@example.com',
    'books': [
        {
            'name': 'Foundation',
            'isbn': 1,
        },
        {
            'name': 'Foundation and Empire',
            'isbn': 2,
        },
    ],
})

User(name='Isaac Asimov', email='isaac.asimov@example.com', books=[Book(name='Foundation', isbn=1), Book(name='Foundation and Empire', isbn=2)])

Вложенные объекты данных намного проще описывать, чем в marshmallow. В marshmallow это нужно использовать fields.Nested

***

### Типы данных для валидации (Field types)

Pydantic поддерживает:
* стандартные типы данных языка Python
* типы и конструкции из модуля typing
* пользовательские типы данных

#### Типы и конструкции из модуля typing

Пример с использованием Optional из typing и EmailStr тип из модуля pydantic:

In [18]:
from typing import Optional
from pydantic import BaseModel, EmailStr


class UserSchema(BaseModel):
    name: Optional[str]
    email: EmailStr

Корректные имя и email:

In [19]:
UserSchema(**{
    'name': 'John Doe',
    'email': 'john.doe@example.com',
})

UserSchema(name='John Doe', email='john.doe@example.com')

Имя не известно, известен только email. В таком случае данные корректны:

In [52]:
UserSchema(**{
    'name': None,
    'email': 'john.doe@example.com',
})

UserSchema(name=None, email='john.doe@example.com')

Некорректный email:

In [20]:
UserSchema(**{
    'name': 'John Doe',
    'email': 'john.doe',
})

ValidationError: 1 validation error for UserSchema
email
  value is not a valid email address (type=value_error.email)

***

Пример с использованием Union из typing:

In [22]:
from typing import Union
from pydantic import BaseModel


class FlatSchema(BaseModel):
    floor: Union[int, str]

Переменная **floor** является **строкой**. Такие данные корректны: 

In [24]:
FlatSchema(floor='underground')

FlatSchema(floor='underground')

Переменная **floor** является **числом**. Такие данные тоже корректны:

In [25]:
FlatSchema(floor=73)

FlatSchema(floor=73)

***

Вместо **Union\[int, str\]**, укажем **Union\[str, int\]**

In [27]:
from typing import Union
from pydantic import BaseModel

    
class FlatSchema(BaseModel):
    floor: Union[str, int]

Pydantic приведет 73 к строке, потому-что Pydantic  идет последовательно по типам из Union и пытается привести данные к заданому типу.  

In [29]:
FlatSchema(floor=73)

FlatSchema(floor='73')

***

#### Пользовательские типы данных

Для создания своих собственных типов данных необходимо определить пользовательский класс с classmethod **\_\_get_validators\_\_**. Этот classmethod будет вызван, чтобы получить валидаторы для парсинга и валидации входных данных в классе модели.

В пользовательском типе **Floor** этаж должен быть целым числом, если где-то подземный этаж указан, как "underground", то мы должны заменить эту строку на число **-1**

In [48]:

class Floor:
    floor_names = {
            'underground': -1,
        }
    
    @classmethod
    def validate(cls, v):
        if v in cls.floor_names:
            v = cls.floor_names[v]
        if isinstance(v, int):
            return v
        raise ValueError(f'int or underground string is expected instead of {v}')
        
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

Используем **Floor**, как пользовательский тип данных для переменной  **floor_number**

In [49]:
from pydantic import BaseModel

class Flat(BaseModel):
    floor_number: Floor

floor_number=1, данные остаются неизменными:

In [34]:
Flat(floor_number=1)

Flat(floor_number=1)

floor_number='underground', после валидации **floor_number** становится равным **-1**:

In [40]:
Flat(floor_number='underground')

Flat(floor_number=-1)

В значение переменной **floor_number** была опечатка. Будет выведена ошибка:

In [41]:
Flat(floor_number='underground-1')

ValidationError: 1 validation error for Flat
floor_number
  int or underground string is expected instead of underground-1 (type=value_error)

***

#### Пример с использованием нескольких валидаторов в пользовательском классе

В Азии некоторые числа являются несчастливыми, например это числа 4, 14, 74. Мы собираемся запретить в пользовательском классе использовать несчастливые числа в качестве номеров этажей, но по преждему этаж 'underground' должен записываться, как этаж с номером -1.
<img src="./images/floor-numbers.jpeg" width="100">

Для этого в предыдущий класс **Floor** необходимо добавить дополнительный валидатор **is_luck** и указать его в функции **\_\_get_validators\_\_**. 

❗️

При определении пользовательского типа данных в classmethod **\_\_get_validators\_\_** необходимо использовать **yield** вместо **return**, так как pydantic ожидает здесь генератор. Это позволяет использовать один или несколько валидаторов, которые будут вызываться для проверки ввода, каждый валидатор получит в качестве ввода значение, возвращенное предыдущим валидатором.

In [50]:
class AsianFloor:
    floor_names = {
            'underground': -1,
        }
    
    @classmethod
    def validate(cls, v):
        if v in cls.floor_names:
            v = cls.floor_names[v]
        if isinstance(v, int):
            return v
        raise ValueError(f'int type or underground string is expected instead of {v}')
        
    @classmethod
    def is_luck(cls, v):
        unluck_numbers = {4, 14, 74}
        if v in unluck_numbers:
            raise ValueError(f'The {v} floor is impossible, so is {unluck_numbers}')
        return v
    
    @classmethod
    def __get_validators__(cls):
        yield cls.validate
        yield cls.is_luck

Модель данных, где типом для переменной **floor_number** является класс **AsianFloor**

In [51]:
from pydantic import BaseModel

class Flat(BaseModel):
    floor_number: AsianFloor

Число 10 разрешено для нумерации этажей

In [53]:
Flat(floor_number=10)

Flat(floor_number=10)

Число 4 несчастливое и оно запрещено для нумерации этажей

In [54]:
Flat(floor_number=4)

ValidationError: 1 validation error for Flat
floor_number
  The 4 floor is impossible, so is {74, 4, 14} (type=value_error)

### Сonstraining  types (Ограничение значений базовых типов данных)

Значения различных стандартных типов может быть ограничены используя функции con* типа. 
Например:
 * constr для str
 * conint для int
 * etc


Ограничим количество этажей от 0 до 200. Для этого используем функцию **conint**, с параметрами ge=0, lt=200

In [56]:
from pydantic import BaseModel, conint

class Flat(BaseModel):
    floor_number: conint(ge=0, lt=200)

10 этаж может существовать

In [117]:
Flat(floor_number=10)

Flat(floor_number=10)

Но не может быть этажа с номером 201

In [118]:
Flat(floor_number=201)

ValidationError: 1 validation error for Flat
floor_number
  ensure this value is less than 200 (type=value_error.number.not_lt; limit_value=200)

***

## Написание собственных валидаторов для модели

In [22]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class FloorSchema(BaseModel):
    id: UUID
    floor_number: int
    
    @validator('floor_number')
    def validate_floor_number(cls, value: int) -> int:
        unluck_numbers = {4, 14, 74}
        if value in unluck_numbers:
            raise ValueError(f'The {value} floor is impossible, so is {unluck_numbers}')
        return value

10 этаж существует:

In [23]:
FloorSchema(id=uuid4(), floor_number=10)

FloorSchema(id=UUID('b9cc9468-9673-4246-b387-9a21f91e5d10'), floor_number=10)

4 этаж не существует:

In [24]:
FloorSchema(id=uuid4(), floor_number=4)

ValidationError: 1 validation error for FloorSchema
floor_number
  The 4 floor is impossible, so is {74, 4, 14} (type=value_error)

***

### Валидация нескольких полей вместе

In [28]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class FloorSchema(BaseModel):
    id: UUID
    floor_number: int
    count_rooms: int
    owner: str
    
    @validator('count_rooms')
    def validate_count_rooms(cls, value, values) -> int:
        if value > values.get('floor_number'):
            raise ValueError(f'The number of rooms must be less than or equal to the floor number.')
        return value

5 этаж, количество комнат 5

In [30]:
FloorSchema(id=uuid4(), floor_number=5, count_rooms=5, owner='John Doe')

FloorSchema(id=UUID('b94bc564-7d45-46ea-8a11-d19b133bbb55'), floor_number=5, count_rooms=5, owner='John Doe')

5 этаж, количество комнат 10

In [ ]:
FloorSchema(id=uuid4(), floor_number=5, count_rooms=10, owner='John Doe')

***

❗️ Нижестоящие переменные модели не доступны при валидации текущей переменной

In [31]:
from uuid import UUID, uuid4
from pydantic import BaseModel, validator


class FloorSchema(BaseModel):
    id: UUID
    floor_number: int
    count_rooms: int
    owner: str
    
    @validator('count_rooms')
    def validate_count_rooms(cls, value, values) -> int:
        print(f'values={values}')
        if value > values.get('floor_number'):
            raise ValueError(f'The number of rooms must be less than or equal to the floor number.')
        return value

In [32]:
FloorSchema(id=uuid4(), floor_number=5, count_rooms=5, owner='John Doe');

values={'id': UUID('229417e3-fa6c-4f43-803b-bb06b63f84c2'), 'floor_number': 5}


***

### Исключения в валидаторах

Pydantic поднимает ValidationError всякий раз, когда обнаруживает ошибку в проверяемых данных

In [82]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
        

User(name=[])

ValidationError: 1 validation error for User
name
  str type expected (type=type_error.str)

***

❗️
Валидаторы реализованные программистом, не должны поднимать ValidationError сами, вместо этого нужно использовать 
* ValueError
* TypeError
* AssertionError 
* или подклассы ValueError или TypeError, 

так как эти исключения pydantic словит и будет использовать для ValidationError.

В конечном счете одно исключение будет вызвано и ValidationError будет содержать информацию обо всех ошибках и описание как они произошли.

***

### Получение сообщений об ошибках

e.json() - возвращает ошибки в виде json. Удобно использовать как response в web-приложениях. Pydantic используется в Fast-API

In [35]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
        

try:
    User(name=[])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "name"
    ],
    "msg": "str type expected",
    "type": "type_error.str"
  }
]


e.errors() - возвращает список найденных ошибок во входных данных

In [89]:
try:
    User(name=[])
except ValidationError as e:
    print(e.errors())

[{'loc': ('name',), 'msg': 'str type expected', 'type': 'type_error.str'}]


str(e) - возвращает ошибки в читаемом формате

In [90]:
try:
    User(name=[])
except ValidationError as e:
    print(e)    # print(str(e))

1 validation error for User
name
  str type expected (type=type_error.str)


### Import data

parse_obj -  на вход принимает dict, а не именованные аргументы. Если переданный объект не является словарем, будет поднято исключение ValidationError. 

In [47]:
from pydantic import BaseModel

class UserSchema(BaseModel):
    name: str
    email: str
    age: int


user_data = {
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'age': 30,
}
UserSchema.parse_obj(user_data)

UserSchema(name='John Doe', email='john.doe@example.com', age=30)

parse_raw -  принимает строку или байты и парсит ее как json

In [48]:
import json
user_json = json.dumps(user_data)
UserSchema.parse_raw(user_json)

UserSchema(name='John Doe', email='john.doe@example.com', age=30)

parse_file - принимает путь файла, читает его и парсит его.

#### Export data

dict() выводит данные объекта в виде словаря

In [49]:
user = UserSchema(**user_data)
user.dict()

{'name': 'John Doe', 'email': 'john.doe@example.com', 'age': 30}

json() выводит данные объекта в виде json

In [50]:
user.json()

'{"name": "John Doe", "email": "john.doe@example.com", "age": 30}'

Исключение некоторых полей из вывода данных:

In [52]:
user.json(exclude={'email'})

'{"name": "John Doe", "age": 30}'

In [53]:
user.dict(exclude={'email'})

{'name': 'John Doe', 'age': 30}

Множество инструментов для сериализации и десериализации данных, делает pydantic удобным для использования в web-приложениях

***

### Валидация данных с избыточной информацией

Иногда из различных сервисов поступают избыточные данные. С помощью параметра **extra** класса **Config** можно управлять поведением модели с избыточными данными.


* extra='ignore' - избыточные данные будут игнорироваться (является значением по умолчанию)
* extra='allow' - избыточные данные будут добавлены, но не будут валидироваться
* extra='forbid' - в случае если были переданы избыточные данные, то будут выведена ошибка

In [73]:
from pydantic import BaseModel
from uuid import UUID, uuid4


class User(BaseModel):
    id: UUID
    username: str
        
    class Config:
        extra='allow'

In [74]:
User(**{'username': 'admin', 'id': uuid4(), 'location': 'Moscow'})

User(id=UUID('aa8c98ed-f479-4a36-9afa-0319bba06b46'), username='admin', age=30, location='Moscow')

По умолчанию избыточные данные будут проигнорированы:

In [54]:
from pydantic import BaseModel
from uuid import UUID, uuid4


class User(BaseModel):
    id: UUID
    username: str

In [55]:
User(**{'username': 'admin', 'id': uuid4(), 'age': 30, 'location': 'Moscow'})

User(id=UUID('6dd4e617-f24c-4b6e-8617-ca2254f4fde1'), username='admin')

***

#### Aliases
alias указывает pydantic по какому ключу искать нужные поля во входных данных. Может быть очень удобно, когда данные поступают из других сервисов или баз данных. Например в сервисе используется **CamelCase**, но в обработанных данных мы хотим использовать **snake_case**

In [56]:
import datetime

from typing import Union
from pydantic import BaseModel, Field



class PostSchema(BaseModel):
    user: str
    text: str
    last_modified_data: datetime.datetime = Field(alias='LastModifiedData')

In [57]:
post = {
    'user': 'admin',
    'text': 'Hello word!',
    'LastModifiedData': datetime.datetime(year=2021, month=5, day=4)
}

PostSchema(**post)

PostSchema(user='admin', text='Hello word!', last_modified_data=datetime.datetime(2021, 5, 4, 0, 0))

***

## Наследование


Наследование работает так же как в python

In [7]:
from typing import Optional
from pydantic import BaseModel, EmailStr


class Person(BaseModel):
    email: EmailStr
    name: Optional[str]
        

class Customer(Person):
    total_spending: Optional[int] = 0

In [8]:
Customer(email='john.doe@example.com', total_spending=200)

Customer(email='john.doe@example.com', name=None, total_spending=200)

In [9]:
Customer('john.doe@example.com', 200)

TypeError: __init__() takes exactly 1 positional argument (3 given)

На первый взгляд не совсем понятно, почему произошла ошибка. С помощью \_\_fields\_\_ посмотрим какие аргументы ожидаются

In [18]:
Customer.__fields__

{'email': ModelField(name='email', type=EmailStr, required=True),
 'name': ModelField(name='name', type=Optional[str], required=False, default=None),
 'total_spending': ModelField(name='total_spending', type=Optional[int], required=False, default=0)}

В данном случае есть несколько правильных способов передавать значения аргументов:


1. parse_raw

In [21]:
Customer.parse_raw('{"email":"foo@bar.com", "total_spending":200}')

Customer(email='foo@bar.com', name=None, total_spending=200)

2. передача данных по именованным аргументам

In [22]:
Customer(email='foo@bar.com', total_spending=200)

Customer(email='foo@bar.com', name=None, total_spending=200)

***

## Использование pydantic для чтение переменных окружения

Задаем переменные окружения: 

In [10]:
%set_env APP_HOST=127.0.0.1

env: APP_HOST=127.0.0.1


In [11]:
%set_env APP_PORT=8000

env: APP_PORT=8000


In [17]:
!set | grep APP_

APP_HOST=127.0.0.1
APP_PORT=8000
ZSH_EXECUTION_STRING='set | grep APP_'


Класс загрузки переменных окружения должен быть унаследован от BaseSettings. В классе **Config** с помощью переменной **env_prefix** можно указать префикс названий переменных окружения

In [18]:
from pydantic import BaseSettings


class AppSettings(BaseSettings):
    host: str
    port: int
        
    class Config:
        env_prefix = 'APP_'

In [19]:
app_settings = AppSettings()
app_settings

AppSettings(host='127.0.0.1', port=8000)

In [20]:
app_settings.host

'127.0.0.1'

***

# Итог


    ✅ Pydantic - удобный и полезный инструмент для валидации данных. 
    ✅ Схема задается в виде класса, а не в виде словарей, что упрощает описание схемы данных
    ✅ Нет необходимости изучать дополнительный язык типов данных. Используется стандартные типы
    ✅ Также можно использовать типы данных из модуля typing и реализовывать собственные типы данных
    ✅ Можно возращать валидированный экземпляр класса
    ✅ Если нужно вернуть экземпляр класса, то нет необходимости реализовать дополнительный класс
    ✅ Если в существующем проекте используются python dataclass, то очень легко добавить валидацию данных с помощью pydantic
    ✅ Легко выполнять сериализацию и десериализацию данных
    ✅ Ошибки при валидации данных подробно описаны и их легко вывести в json-формате, что позволяет выдавать ошибки, как response в web-приложениях

***

Документация пакета pydantic https://pydantic-docs.helpmanual.io/

***

# Спасибо за внимание

***